## Part 1: Pulse Rate Algorithm

### Contents
Fill out this notebook as part of your final project submission.

**You will have to complete both the Code and Project Write-up sections.**
- The [Code](#Code) is where you will write a **pulse rate algorithm** and already includes the starter code.
   - Imports - These are the imports needed for Part 1 of the final project. 
     - [glob](https://docs.python.org/3/library/glob.html)
     - [numpy](https://numpy.org/)
     - [scipy](https://www.scipy.org/)
- The [Project Write-up](#Project-Write-up) to describe why you wrote the algorithm for the specific case.


### Dataset
You will be using the **Troika**[1] dataset to build your algorithm. Find the dataset under `datasets/troika/training_data`. The `README` in that folder will tell you how to interpret the data. The starter code contains a function to help load these files.

1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

### Code

In [90]:
import glob

import numpy as np
import scipy as sp
import scipy.signal
import scipy.io
import time
import matplotlib.pyplot as plt

import mpld3
mpld3.enable_notebook()


def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]


def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

In [315]:
def test_function(idx, figures=False, specgram=False):
    '''
    Top-level function to test code.
    Runs the pulse rate algorithm on one subject of the Troika dataset
    and prints some metrics and dsiplay figures.
    Parameters: - idx: positive int, index of subject to evaluate 
                - figures: boolean, option to show signals' plot
                - specgram: boolean, option to show signals' spectogram
    Returns: None
    '''
    # Retrieve a subject's dataset file
    data_fls, ref_fls = LoadTroikaDataset()
    data, ref = data_fls[idx], ref_fls[idx]
    # Run the pulse rate algorithm on one trial of the dataset
    errors, confidence = RunPulseRateAlgorithm(data, ref, figures=figures, specgram=specgram)
    
    print("Mean error: %.3f"%(AggregateErrorMetric(errors, confidence)))
    
def figure_params(kind=None):
    '''
    Auxiliary function setting some parameters
    to plot figures.
    Parameters: - kind: type of figure to show, only 'specgram' available.
    Returns: None
    '''  
    if (kind == "specgram"):
        plt.ylabel("Frequency (Hz)", size=16)
        plt.ylim(40/60, 240/60)
    else:
        plt.ylabel("Signal (a.u.)", size=16)
        plt.legend(fontsize=15)
    
    plt.xlabel("Time (minutes)", size=16)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.show()

def plot_data(ppg, accx, accy, accz, fs, specgram=False, description='Raw'):
    '''
    Function to plot and show the signals ppg, accx, accy, accz.
    Inputs: - ppg: Numpy array of PPG signal
            - accx: Numpy array of PPG signal
            - accy: Numpy array of PPG signal
            - accz: Numpy array of PPG signal
            - fs: sampling rate in Hz, assumed equal for all signals
            - specgram: boolean, option to show signals' spectogram 
            - description: string describing signal
    Returns: None
    '''
    figsize = (10,6)
    # time stamps
    ts = np.arange(len(ppg)) / fs / 60
    print("---------------------------------------")
    print(description + ' signal:')
    print("---------------------------------------")
    # plot PPG signal
    plt.figure(figsize=figsize)
    plt.plot(ts, ppg, label="PPG")
    plt.title(description + " photoplethysmogram signal", size=20)
    figure_params()
    # plot accelerometers signal
    plt.figure(figsize=figsize)
    plt.plot(ts, accx, label="x")
    plt.plot(ts, accy, label="y")
    plt.plot(ts, accz, label="z")
    plt.title(description + " accelerometers signal", size=20)
    figure_params()
    plt.figure(figsize=figsize)
    # plot total acceleration amplitude
    acc = np.sqrt(np.square(accx) + np.square(accy) + np.square(accz))
    plt.plot(ts, acc, label="Total magnitude")
    plt.title(description + " accelerometer total amplitude signal", size=20)
    figure_params()
    # plot specgrams if selected
    if specgram:
        kind = 'specgram'
        # PPG signal
        plt.figure(figsize=figsize)
        plt.specgram(ppg, Fs=fs, NFFT=8 * fs, noverlap=6 * fs, xextent=(0, len(ppg) / fs / 60))
        plt.title(description + " photoplethysmogram signal", size=20)
        figure_params(kind)
        # accx signal
        plt.figure(figsize=figsize)
        plt.specgram(accx, Fs=fs, NFFT=8 * fs, noverlap=6 * fs, xextent=(0, len(ppg) / fs / 60))
        plt.title(description + " accelerometer signal: x-direction", size=20)
        figure_params(kind)
        # accy signal
        plt.figure(figsize=figsize)
        plt.specgram(accy, Fs=fs, NFFT=8 * fs, noverlap=6 * fs, xextent=(0, len(ppg) / fs / 60))
        plt.title(description + " accelerometer signal: x-direction", size=20)
        figure_params(kind)
        # accz signal
        plt.figure(figsize=figsize)
        plt.specgram(accz, Fs=fs, NFFT=8 * fs, noverlap=6 * fs, xextent=(0, len(ppg) / fs / 60))
        plt.title(description + " accelerometer signal: x-direction", size=20)
        figure_params(kind)
        # total acc signal
        plt.figure(figsize=figsize)
        plt.specgram(acc, Fs=fs, NFFT=8 * fs, noverlap=6 * fs, xextent=(0, len(ppg) / fs / 60))
        plt.title(description + " accelerometer total magnitude signal", size=20)
        figure_params(kind)
        
def BandpassFilter(signal, pass_band, fs):
    '''
    Applies a bandpass filter to the signal.
    Parameters: - signal: numpy array, input signal
                - pass_band: tuple (frequency_min, frequency_max), frequency pass band,
                             components outside the tuple range will be removed.
                - fs: sampling rate in Hz 
    Returns: - Filtered signal as a nump.array
    '''
    b, a = sp.signal.butter(3, pass_band, btype='bandpass', fs=fs)
    
    return sp.signal.filtfilt(b, a, signal)

def extract_frequency(freqs, ppg_fft_amp, acc_fft_amp, accx_fft_amp, accy_fft_amp, accz_fft_amp):
    '''
    Returns the frequency and position in freqs array corresponding to the 
    larger amplitude in ppg_fft_amp that is also not present in acc_fft_amp.
    Parameters: - freqs: numpy array of positive values, frequencies of FFT
                - ppg_amp: numpy array, corresponding PPG's FFT amplitudes
                - acc_amp: numpy array, corresponding acceleration's FFT amplitudes
    Returns: - freq: the corresponding frequency
             - idx: index corresponding to freq
    '''
    # indexes for input arrays
    idxs = list(range(len(freqs)))
    # sort amplitudes in descending order 
    # by keeping their position in original array
    sort_ppg_fft_amp = sorted(list(zip(ppg_fft_amp, idxs)), reverse=True)
    sort_acc_fft_amp = sorted(list(zip(acc_fft_amp, idxs)), reverse=True)
    sort_accx_fft_amp = sorted(list(zip(accx_fft_amp, idxs)), reverse=True)
    sort_accy_fft_amp = sorted(list(zip(accy_fft_amp, idxs)), reverse=True)
    sort_accz_fft_amp = sorted(list(zip(accz_fft_amp, idxs)), reverse=True)

    # for ppg_tuple, acc_tuple in zip(sort_ppg_fft_amp, sort_acc_fft_amp):
    #     # if different frequency/index in ppg and acc signals, 
    #     # then return values
    #     if (ppg_tuple[1] != acc_tuple[1]):
    #         idx = ppg_tuple[1]
    #         freq = freqs[idx]
    #    
    #        return freq, idx
    for ppg_tuple, acc_tuple, accx_tuple, accy_tuple, accz_tuple \
      in zip(sort_ppg_fft_amp, sort_acc_fft_amp, sort_accx_fft_amp, sort_accy_fft_amp, sort_accz_fft_amp):
        if (ppg_tuple[1] != acc_tuple[1]) & (ppg_tuple[1] != accx_tuple[1]) & (ppg_tuple[1] != accx_tuple[1]) \
          & (ppg_tuple[1] != accy_tuple[1]) & (ppg_tuple[1] != accz_tuple[1]) :
            idx = ppg_tuple[1]
            freq = freqs[idx]
            
            return freq, idx
        
    # if no solution in loop return ppg's frequency of max amplitude
    idx = sort_ppg_fft_amp[0][1]
    freq = freqs[idx]
    
    return freq, idx

def RunPulseRateAlgorithm(data_fl, ref_fl, figures="", specgram=False):
    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    ppg = ppg - np.mean(ppg)
    accx = accx - np.mean(accx)
    accy = accy - np.mean(accy)
    accz = accz - np.mean(accz)
    # calculate total acceleration
    acc = np.sqrt(np.square(accx) + np.square(accy) + np.square(accz))
    # Load data from reference file
    references = sp.io.loadmat(ref_fl)['BPM0'].flatten()

    # Compute pulse rate estimates and estimation confidence.
    fs = 125 # Hz according to data Readme.pdf
    freq_min = 40 / 60 # minimal frequency from BPM to Hz
    freq_max = 240 / 60 # upper limit frequency from BPM to Hz
    bp_lims = (freq_min, freq_max)
    window_shift = 2 * fs # num samples in 2 seconds
    # window length to match window length of references
    window_length = np.int(np.around((len(ppg) - len(references) * window_shift)))
    
    # option to plot raw signals
    if ("raw" in figures):
        plot_data(ppg, accx, accy, accz, fs, specgram=specgram)
        
    # filtering signals bewteen freq_min and freq_max
    bp_ppg = BandpassFilter(ppg, bp_lims, fs)
    bp_accx = BandpassFilter(accx, bp_lims, fs)
    bp_accy = BandpassFilter(accy, bp_lims, fs)
    bp_accz = BandpassFilter(accz, bp_lims, fs)
    bp_acc = BandpassFilter(acc, bp_lims, fs)
    
    if ("filtered" in figures):
        plot_data(bp_ppg, bp_accx, bp_accy, bp_accz, fs, specgram=specgram, description="Filtered")
        
    estimated_pulses = []
    confidences = []
    for i in range(0, len(bp_ppg) - window_length, window_shift):
        windowed_ppg = bp_ppg[i:i + window_length]
        windowed_acc = bp_acc[i:i + window_length]
        windowed_accx = bp_accx[i:i + window_length]
        windowed_accy = bp_accy[i:i + window_length]
        windowed_accz = bp_accz[i:i + window_length]
        # calculate FFTs, and keep only frequencies
        # between freq_min and freq_max
        freqs = np.fft.rfftfreq(len(windowed_ppg), 1 / fs)
        filter_freqs = (freqs > freq_min) & (freqs < freq_max)
        
        ppg_fft_amp = np.abs(np.fft.rfft(windowed_ppg))[filter_freqs]
        acc_fft_amp = np.abs(np.fft.rfft(windowed_acc))[filter_freqs]
        accx_fft_amp = np.abs(np.fft.rfft(windowed_accx))[filter_freqs]
        accy_fft_amp = np.abs(np.fft.rfft(windowed_accy))[filter_freqs]
        accz_fft_amp = np.abs(np.fft.rfft(windowed_accz))[filter_freqs]
        freqs = freqs[filter_freqs]
        
        # get frequency of max amplitude and its corresponding index
        freq, idx = extract_frequency(freqs, ppg_fft_amp, acc_fft_amp, accx_fft_amp, accy_fft_amp, accz_fft_amp)
        confidence = np.sum(ppg_fft_amp[idx-2:idx+2]) / np.sum(ppg_fft_amp)
        
#         # index of max amplitude
#         idx_max_ppg = np.argmax(ppg_fft_amp)
#         idx_max_acc = np.argmax(acc_fft_amp)
#         # if dominant accelerometer frequency
#         # different from dominant PPG frequency
#         if idx_max_acc != idx_max_ppg:
#             freq = freqs[idx_max_ppg]
#         else:
#             idx_max_ppg_left = np.argmax(ppg_fft_amp[:idx_max_ppg])
#             idx_max_acc_left = np.argmax(acc_fft_amp[:idx_max_acc])
#             idx_max_ppg_right = np.argmax(ppg_fft_amp[idx_max_ppg+1:])
#             idx_max_acc_right = np.argmax(acc_fft_amp[idx_max_acc+1:])
#             if (idx_max_ppg_left<idx_max_ppg_right):
#                 idx_max_ppg = idx_max_ppg_right
#             else:
#                 idx_max_ppg = idx_max_ppg_left
#             if (idx_max_acc_left<idx_max_acc_right):
#                 idx_max_acc = idx_max_acc_right
#             else:
#                 idx_max_pacc = idx_max_acc_left
#             if idx_max_acc != idx_max_ppg:
#                 freq = freqs[idx_max_ppg]
#             else:
#                 freq = 0
#         confidence = (ppg_fft_amp[idx_max_ppg] - acc_fft_amp[idx_max_ppg]) / ppg_fft_amp[idx_max_ppg]
        
        
            
        estimated_pulses.append(freq * 60)
        confidences.append(confidence)
        
        
    estimated_pulses = np.array(estimated_pulses)
    confidences = np.array(confidences)
    #print(np.percentile(confidences, 10))
    #print(estimated_pulses)
    #print(confidences)

    len_estimated_pulses = len(estimated_pulses)
    len_references = len(references)
    min_len = np.min((len_estimated_pulses, len_references))
    errors = np.abs(estimated_pulses[:min_len] - references[:min_len])
    confidences = confidences[:min_len]
    #print(AggregateErrorMetric(errors, confidences))
    #print(estimated_pulses/60)
    #print(estimated_pulses)
    #print(references)
    print(np.mean(errors))
    
    
    # Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.
    #errors, confidence = np.ones(100), np.ones(100)  # Dummy placeholders. Remove
    return errors, confidences

#test_function(0, "", True)
Evaluate()

22.957315127540884
11.212707555669358
5.28545826664741
13.503097327510124
8.656049315068495
10.198145701806633
20.689999392898052
25.006042264290645
10.382269913605231
26.65317874720358
23.941543356643358
18.02640056406124


15.004133604918296

In [135]:
freqs = np.fft.rfftfreq(10*125, 1 / 125)
freqs = freqs[(freqs>40 / 60) & (freqs<(240/60))]
freqs

array([0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9,
       2. , 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3. , 3.1, 3.2,
       3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9])

In [261]:
90 / 60

1.5

In [263]:
40 / 60

0.6666666666666666

In [264]:
240 / 60

4.0

In [223]:
sorted(list(zip(range(10), range(10,0,-1))), reverse=True)

[(9, 1),
 (8, 2),
 (7, 3),
 (6, 4),
 (5, 5),
 (4, 6),
 (3, 7),
 (2, 8),
 (1, 9),
 (0, 10)]

-----
### Project Write-up

Answer the following prompts to demonstrate understanding of the algorithm you wrote for this specific context.

> - **Code Description** - Include details so someone unfamiliar with your project will know how to run your code and use your algorithm. 
> - **Data Description** - Describe the dataset that was used to train and test the algorithm. Include its short-comings and what data would be required to build a more complete dataset.
> - **Algorithhm Description** will include the following:
>   - how the algorithm works
>   - the specific aspects of the physiology that it takes advantage of
>   - a describtion of the algorithm outputs
>   - caveats on algorithm outputs 
>   - common failure modes
> - **Algorithm Performance** - Detail how performance was computed (eg. using cross-validation or train-test split) and what metrics were optimized for. Include error metrics that would be relevant to users of your algorithm. Caveat your performance numbers by acknowledging how generalizable they may or may not be on different datasets.

Your write-up goes here...

-----
### Next Steps
You will now go to **Test Your Algorithm** (back in the Project Classroom) to apply a unit test to confirm that your algorithm met the success criteria. 